In [1]:
%reload_ext dotenv
%dotenv .env

In [2]:
import overpy
import sqlite3
import os

from queue import PriorityQueue
from collections import defaultdict
from folium import Map, PolyLine

from classes import GraphNode

api = overpy.Overpass()
connection = sqlite3.connect(os.environ.get('DATABASE_NAME'))
cursor = connection.cursor()

#### Execute query using Overpass Turbo

In [3]:
tram_railways_query_result = api.query("""
    [out:json];
    area["name"="Kraków"]->.search_area;
    (
        way["railway"="tram"]["oneway"="yes"](area.search_area);
        >;
    );
    out geom;
""")

#### Create a graph of nodes and edges

In [4]:
graph: dict[int, GraphNode] = {}

for item in tram_railways_query_result.ways:
    nodes = item.get_nodes()

    for node in nodes:
        if node.id not in graph:
            graph[node.id] = GraphNode(node.id, node.lat, node.lon)

    for start, end in zip(nodes, nodes[1:]):
        graph[start.id].edges.append(graph[end.id])

list(graph.values())

[GraphNode(id=321437325, latitude=Decimal('50.0724838'), longitude=Decimal('20.0376671')),
 GraphNode(id=213605067, latitude=Decimal('50.0725312'), longitude=Decimal('20.0373188')),
 GraphNode(id=3071452252, latitude=Decimal('50.0663517'), longitude=Decimal('19.9585038')),
 GraphNode(id=3071450411, latitude=Decimal('50.0664205'), longitude=Decimal('19.9584619')),
 GraphNode(id=3071450414, latitude=Decimal('50.0665347'), longitude=Decimal('19.9583819')),
 GraphNode(id=3071450415, latitude=Decimal('50.0666227'), longitude=Decimal('19.9583062')),
 GraphNode(id=3071450416, latitude=Decimal('50.0666625'), longitude=Decimal('19.9582670')),
 GraphNode(id=3071450417, latitude=Decimal('50.0667182'), longitude=Decimal('19.9582049')),
 GraphNode(id=3071450418, latitude=Decimal('50.0667704'), longitude=Decimal('19.9581340')),
 GraphNode(id=276371937, latitude=Decimal('50.0668129'), longitude=Decimal('19.9580691')),
 GraphNode(id=3071447313, latitude=Decimal('50.0668912'), longitude=Decimal('19.957

#### Get a list of routes between stops

In [5]:
cursor.execute("""
    SELECT DISTINCT
        tram_passage_stops.tram_stop_id AS start_stop_id,
        TPS.tram_stop_id AS end_stop_id
    FROM tram_passage_stops
        JOIN tram_passage_stops TPS ON tram_passage_stops.tram_passage_id = TPS.tram_passage_id
    WHERE tram_passage_stops.stop_index = TPS.stop_index - 1
""")

stops = cursor.fetchall()
len(stops)

419

#### Get shortest route between nodes

In [6]:
def route_between_nodes(graph: dict[int, GraphNode], start_node_id: int, end_node_id: int):
    priority_queue: PriorityQueue[tuple[float, GraphNode]] = PriorityQueue()
    parents: dict[int, GraphNode] = {}
    distances: dict[int, float] = defaultdict(lambda: float("inf"))

    priority_queue.put((0, graph[start_node_id]))
    distances[start_node_id] = 0.0

    while priority_queue.not_empty:
        current_distance, root = priority_queue.get()

        if current_distance > distances[end_node_id]:
            break

        for node in graph[root.id].edges:
            new_distance = distances[root.id] + root.distance_to(node)

            if distances[node.id] > new_distance:
                distances[node.id] = new_distance
                parents[node.id] = root
                priority_queue.put((distances[node.id], node))

    route = [graph[end_node_id]]
    while route[-1] != graph[start_node_id]:
        route.append(parents[route[-1].id])
        
    route.append(graph[start_node_id])

    route.reverse()

    return route

#### Calculate routes between stops

In [7]:
i = 0

In [8]:
route = route_between_nodes(graph, *stops[i])
print(i)
i += 1

latitudes, longitudes = [item.latitude for item in route], [item.longitude for item in route]

folium_map = Map(
    location=(
        (max(latitudes) - min(latitudes)) / 2 + min(latitudes),
        (max(longitudes) - min(longitudes)) / 2 + min(longitudes),
    ),
    zoom_start=17
)

folium_map.add_child(PolyLine([item.coordinates for item in route]))

folium_map

0
